In [1]:
import pandas as pd 
df_raw = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/data Row/mom.csv', index_col=0)

In [4]:
df_raw = df_raw.drop(df_raw.iloc[:,8:],axis=1)

In [5]:
df_raw

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트
1,2022-12-23,10150,10600,11200,10100,8180753,86595242100,바이오노트
2,2022-12-26,9720,10550,10700,9650,4479667,45206813740,바이오노트
3,2022-12-27,9360,9770,9880,9130,4101930,38365054640,바이오노트
4,2022-12-28,8870,9390,9420,8700,3177379,28837782380,바이오노트
...,...,...,...,...,...,...,...,...
2621629,2022-12-23,3765,3900,3900,3765,33477,127543190,모헨즈
2621630,2022-12-26,3720,3765,3765,3610,30218,111265525,모헨즈
2621631,2022-12-27,3755,3700,3755,3665,17856,66261900,모헨즈
2621632,2022-12-28,3630,3770,3775,3615,25279,93477675,모헨즈


In [18]:
def make_momentum_signal(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        con1 = (df.loc[idx[0]:idx[-1],'종가'].shift(-3))
        con2 = (df.loc[idx[0]:idx[-1],'종가'].shift(-2))
        con3 = (df.loc[idx[0]:idx[-1],'종가'].shift(-1))
        df.loc[idx[0]:idx[-1], 'momentum'] = (con1<con2)&(con2<con3)&(con3<(df.loc[idx[0]:idx[-1], '종가']))
    return df
        

In [19]:
df_test = make_momentum_signal(df_raw)

In [39]:
# market_timming 병합작업
def making_mt(start,end,code):
    from pykrx import stock
    df = stock.get_index_fundamental(start,end,code)
    df = df.drop(['등락률','PER','선행PER','PBR','배당수익률'],axis=1)
    
    ''' code : KOSPI ('1001') KOSDAQ ('2001') '''
    if code=='1001': 
        df['sma20'] = df['종가'].rolling(20).mean()
        df['mt'] = df['종가']>df['sma20']
        df = df.dropna()
        return df
    elif code=='2001':     
        df['sma3'] = df['종가'].rolling(3).mean()
        df['sma5'] = df['종가'].rolling(5).mean()
        df['sma10'] = df['종가'].rolling(10).mean()
        df['mt'] = ((df['종가']>df['sma3'])|(df['종가']>df['sma5'])|(df['종가']>df['sma10']))
        df = df.dropna()
        return df
    else:
        print('올바른 종목 code 입력')

In [55]:
start = df_test[df_test['종목명']=='삼성전자'].head(1)['날짜'].values[0]
end = df_test[df_test['종목명']=='삼성전자'].tail(1)['날짜'].values[0]

In [67]:
df_mt = making_mt(start,end,'2001')
df_mt['날짜']=df_mt.index

#### 거래대금 필터만우선적용..

In [73]:
df_test['vol_condition'] = df_test['누적거래대금'] >=10000000000

#### 최종시그널

In [108]:
def make_buysell(df):
    df['buy'] = (df['vol_condition'])&(df['momentum'])
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        df.loc[idx[0]:idx[-1],'sell'] = df.loc[idx[0]:idx[-1],'buy'].shift(3)
    return df

In [109]:
df_signal = make_buysell(df_test)

In [111]:
df_signal = df_signal.drop('test',axis=1)

In [113]:
df_result = df_signal.dropna()

In [115]:
df_result.isnull().sum()

날짜               0
종가               0
시가               0
최고가              0
최저가              0
누적거래량            0
누적거래대금           0
종목명              0
momentum         0
vol_condition    0
buy              0
sell             0
dtype: int64

In [124]:
code_kospi = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/SRC/current_kospi_codes.csv')
code_kosdaq = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/SRC/current_kosdaq_codes.csv')

In [126]:
code = pd.concat([code_kospi,code_kosdaq])

In [133]:
code.rename(columns={'회사명':'종목명'},inplace=True)

In [134]:
code

,종목명,종목코드,상장일
0,바이오노트,377740,2022-12-22
1,유니드비티플러스,446070,2022-11-28
2,LX세미콘,108320,2022-11-03
3,KB스타리츠,432320,2022-10-06
4,쏘카,403550,2022-08-22
...,...,...,...
1607,하이록코리아,13030,1989-12-15
1608,SBI인베스트먼트,19550,1989-09-06
1609,리더스 기술투자,19570,1989-03-14
1610,엠벤처투자,19590,1989-03-14


In [135]:
df_join = df_result.merge(code,on='종목명',how='inner')

In [139]:
df_join.rename(columns={'종목코드':'코드'},inplace=True)

In [145]:
df_join['구매수량']=3

#### 백테스트

In [140]:
df_join

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,momentum,vol_condition,buy,sell,코드,상장일
0,2022-12-27,9360,9770,9880,9130,4101930,38365054640,바이오노트,False,True,False,True,377740,2022-12-22
1,2022-12-28,8870,9390,9420,8700,3177379,28837782380,바이오노트,False,True,False,True,377740,2022-12-22
2,2022-12-29,8580,8890,8920,8450,2059794,17721767120,바이오노트,False,True,False,True,377740,2022-12-22
3,2022-12-01,6330,6470,6530,6320,431925,2764739060,유니드비티플러스,False,False,False,True,446070,2022-11-28
4,2022-12-02,6280,6330,6480,6280,306045,1945031730,유니드비티플러스,False,False,False,False,446070,2022-11-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537788,2022-12-23,3765,3900,3900,3765,33477,127543190,모헨즈,False,False,False,False,6920,1989-01-05
2537789,2022-12-26,3720,3765,3765,3610,30218,111265525,모헨즈,False,False,False,False,6920,1989-01-05
2537790,2022-12-27,3755,3700,3755,3665,17856,66261900,모헨즈,False,False,False,False,6920,1989-01-05
2537791,2022-12-28,3630,3770,3775,3615,25279,93477675,모헨즈,False,False,False,False,6920,1989-01-05


In [119]:
import sys
sys.path.append('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test')
from BackTesting_System.backtesting import BackTesting, Utils

In [142]:
bt1 = BackTesting()
ut = Utils()

In [149]:
bt1.MONEY = 2000000000

In [150]:
bt1.buy_sell(df=df_join,buy='buy',sell='sell')

In [151]:
bt1.update_information()

In [153]:
bt1.JOUNAL['매매구분'].unique()

array(['매수'], dtype=object)

In [154]:
bt1.ECONO_INFORM

,날짜,매수총합,매도총합,매매비용,실현손익
0,2018-02-01,28293477,0.0,39756.0,0.0
1,2018-02-02,27523809,0.0,38678.0,0.0
2,2018-02-05,8506431,0.0,11951.0,0.0
3,2018-02-06,10390860,0.0,14602.0,0.0
4,2018-02-07,3539592,0.0,4974.0,0.0
...,...,...,...,...,...
525,2020-04-24,66900,0.0,94.0,0.0
526,2020-05-18,40950,0.0,58.0,0.0
527,2020-05-28,10620,0.0,15.0,0.0
528,2020-06-05,4650,0.0,7.0,0.0


In [156]:
df_test_mt = df_join.merge(df_mt,on='날짜',how='inner')

ValueError: '날짜' is both an index level and a column label, which is ambiguous.